In [ ]:
import os
HOME = os.environ["HOME"]
os.environ["CARDIAC_GWAS_REPO"] = CARDIAC_GWAS_REPO = f"{HOME}/01_repos/CardiacGWAS"
os.environ["CARDIAC_COMA_REPO"] = CARDIAC_COMA_REPO = f"{HOME}/01_repos/CardiacCOMA/"
os.environ["GWAS_REPO"] = GWAS_REPO = f

MLRUNS_DIR = f"{CARDIAC_COMA_REPO}/mlruns"
#os.chdir(CARDIAC_COMA_REPO)

In [ ]:
import mlflow
from mlflow.tracking import MlflowClient

import os, sys

import torch
import torch.nn.functional as F

from CardiacCOMA.config.cli_args import overwrite_config_items
from CardiacCOMA.config.load_config import load_yaml_config, to_dict
from CardiacCOMA.utils.helpers import get_datamodule, get_lightning_module
from CardiacCOMA.utils.mlflow_helpers import get_model_pretrained_weights
from CardiacCOMA.utils.CardioMesh.CardiacMesh import transform_mesh

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image, display, Markdown, clear_output

import pandas as pd
import shlex
from subprocess import check_output

import pickle as pkl
import pytorch_lightning as pl

from argparse import Namespace
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from IPython import embed
sys.path.insert(0, '..')

from copy import deepcopy
from pprint import pprint

from typing import List
from tqdm import tqdm

import pandas as pd

import pyvista as pv
from ipywidgets import interact, interactive, fixed, interact_manual

from auxiliary import load_data
from auxiliary import get_model_pretrained_weights
import warnings

from image_helpers import merge_pngs

In [ ]:
from image_helpers import merge_pngs

In [ ]:
regions = [
   "chr17_27",
   "chr12_69",
   "chr6_78",
   "chr2_108",
   "chr11_2",
   "chr5_103",
   "chr1_124",
   "chr6_84",
   "chr2_69",
   "chr12_67"
]

In [ ]:
good_runs_df = pd.read_csv(f"{CARDIAC_GWAS_REPO}/results/good_runs.csv")
run_ids = good_runs_df.run_id.to_list()

gwas_harmonized_pattern = "data/other_gwas/preprocessed_files/{prefix}__{phenotype}.tsv"
gwas_selected_snps_pattern = "data/other_gwas/preprocessed_files/{prefix}__{phenotype}__selected_snps.tsv"
COMA_GWAS_SUMMARY = "results/gwas_loci_summary_across_runs.csv"
LOGP_PATH = "results/log10p_for_selected_snps_across_gwas.csv"

In [ ]:
good_runs_df = pd.read_csv(f"{CARDIAC_GWAS_REPO}/results/good_runs.csv")
run_ids = good_runs_df.run_id.to_list()
gwas_loci_summary_across_runs_df = pd.read_csv(COMA_GWAS_SUMMARY)

# get index of best locus/variable
idx = gwas_loci_summary_across_runs_df.groupby(["region"])["P"].transform(min) == gwas_loci_summary_across_runs_df["P"]

best_association_per_region = gwas_loci_summary_across_runs_df[idx].sort_values("region")
best_snps = set(best_association_per_region.SNP)

regions = { 
    f"{assoc[1].region} ({assoc[1].P:.1e})": assoc[1].region 
    for assoc in best_association_per_region.sort_values("P").iterrows() 
} 

del regions["chr6_79 (4.5e-20)"]

_best_association_per_region = best_association_per_region.set_index("region")
# assoc = _best_association_per_region.loc[region]
#run_id, z_variable = assoc.run, assoc.pheno[-4:]

In [ ]:
_best_association_per_region

In [ ]:
exp_id = "1"

In [ ]:
from PIL import Image

In [ ]:
img = Image.open(manhattan_filename)

In [ ]:
for region in tqdm(regions.values()):
  
    assoc = _best_association_per_region.loc[region]
    run_id, z = assoc.run, assoc.pheno[-4:]
  
    df = pd.read_csv(f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/latent_vector.csv").set_index("ID")
          
    z_effect_figs_dir = f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/z_effect_on_shape"
    shapes_filename = f"{z_effect_figs_dir}/{z}.png"
    
    manhattan_filename = f"GWAS__{z}__{exp_id}_{run_id}__manhattan.png"
    manhattan_filename = f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/GWAS_adj_10PCs/figures/{manhattan_filename}"
            
    locuszoom_filename = f"/home/rodrigo/tmp/locuszoom_by_region/{region}__500kb__{z}.png"
                
    
    concat_fig = f"{region}.png"
    # print(shapes_filename, manhattan_filename, locuszoom_filename)
    try:
        
        manhattan_img = Image.open(manhattan_filename)
        locuszoom_img = Image.open(locuszoom_filename)
        shapes_img = Image.open(shapes_filename)
    
        max_x_size = max([ img.size[0] for img in [shapes_img, manhattan_img, locuszoom_img]])
                        
        manhattan_dims = (max_x_size, int(manhattan_img.size[1] * max_x_size // manhattan_img.size[0]))
        manhattan_img = manhattan_img.resize(manhattan_dims)
        
        # locuszoom_ratio = max_x_size / locuszoom_img.size[0]
        locuszoom_dims = (max_x_size, locuszoom_img.size[1] * max_x_size // locuszoom_img.size[0])
        locuszoom_img = locuszoom_img.resize(locuszoom_dims)
        
        # shapes_ratio = max_x_size / shapes_img.size[0]
        shapes_dims = (max_x_size, max_x_size * shapes_img.size[1] // shapes_img.size[0] )
        shapes_img = shapes_img.resize(shapes_dims)
        
        merge_pngs([shapes_filename, manhattan_filename, locuszoom_filename], concat_fig, "vertically")  
        
        display(shapes_img)
        display(manhattan_img)
        display(locuszoom_img)
        
    except FileNotFoundError:
        continue